In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive
#/35GB notebooks/NLP/Embold_Participant_Dataset/

/content/drive/My Drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
train = pd.read_json("embold_train.json").reset_index(drop=True)
train_extra = pd.read_json("embold_train_extra.json").reset_index(drop=True)
train = pd.concat([train,train_extra]).reset_index(drop=True)
test = pd.read_json("embold_test.json").reset_index(drop=True)
sample = pd.read_csv("sample submission.csv")
train.shape,test.shape,sample.shape

((450000, 3), (30000, 2), (30000, 1))

In [ ]:
train.head()

,title,body,label
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0


In [ ]:
train["text"] = train["title"].astype(str)+" "+train["body"].astype(str)
test["text"] = test["title"].astype(str)+" "+test["body"].astype(str)

In [ ]:
train.label.value_counts()

1    207318
0    200481
2     42201
Name: label, dtype: int64

In [ ]:
train['label0']= np.where(train.label==0,1,0)
train['label1']= np.where(train.label==1,1,0)
train['label2']= np.where(train.label==2,1,0)
label_cols= ['label0','label1','label2']

In [ ]:
train['text'].fillna("unknown", inplace=True)
test['text'].fillna("unknown", inplace=True)

##NBSVM

In [ ]:
# import re, string
# re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
# def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
# n = train.shape[0]
# vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
#                min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
#                smooth_idf=1, sublinear_tf=1 )
# trn_term_doc = vec.fit_transform(train['text'])
# test_term_doc = vec.transform(test['text'])

In [ ]:
# def pr(y_i, y):
#     p = x[y==y_i].sum(0)
#     return (p+1) / ((y==y_i).sum()+1)

In [ ]:
# x = trn_term_doc
# test_x = test_term_doc

In [ ]:
# def get_mdl(y):
#     y = y.values
#     r = np.log(pr(1,y) / pr(0,y))
#     m = LogisticRegression(C=3, dual=False,max_iter=10000)
#     x_nb = x.multiply(r)
#     return m.fit(x_nb, y), r

In [ ]:
# preds = np.zeros((len(test), len(label_cols)))

# for i, j in enumerate(label_cols):
#     print('fit', j)
#     m,r = get_mdl(train[j])
#     preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

In [ ]:
# preds  = pd.DataFrame({'label':[i.argmax() for i in preds]})

In [ ]:
# preds.value_counts()

In [ ]:
# preds.to_csv("result1_large_nbsvm.csv")

##BERT

In [ ]:
from tqdm.notebook import tqdm
import tensorflow as tf

In [ ]:
!pip install transformers==2.11.0
# !pip show transformers

In [ ]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-large-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                            truncation=True
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

input_ids = tokenize_sentences(train['text'], tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks = create_attention_masks(input_ids)

In [ ]:
from sklearn.model_selection import train_test_split
labels =  train[label_cols].values

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.1)

train_size = len(train_inputs)
validation_size = len(validation_inputs)

In [ ]:
train_size,validation_size

(405000, 45000)

In [ ]:
BATCH_SIZE = 16
NR_EPOCHS = 1

def create_dataset(data_tuple, epochs=1, batch_size=16, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    if train:
        dataset = dataset.prefetch(1)
    
    return dataset

train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)

In [ ]:
from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Flatten

class BertClassifier(tf.keras.Model):    
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = Dense(num_classes, activation='sigmoid')
        
    @tf.function
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = outputs[1]
        cls_output = self.classifier(cls_output)
                
        return cls_output

model = BertClassifier(TFBertModel.from_pretrained(bert_model_name), len(label_cols))

In [ ]:
import time
from transformers import create_optimizer

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = validation_size // BATCH_SIZE

# | Loss Function
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# | Optimizer (with 1-cycle-policy)
warmup_steps = steps_per_epoch // 3
total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps
optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# | Metrics
train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)
    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables),'1.0')
    train_loss(loss)
    
   

    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
        
@tf.function
def validation_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
                                              
def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
    for epoch in range(epochs):
        print('=' * 50, f"EPOCH {epoch}", '=' * 50)

        start = time.time()

        for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
            train_step(model, token_ids, masks, labels)
            if i % 1000 == 0:
                print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
                for i, label_name in enumerate(label_cols):
                    print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
                    train_auc_metrics[i].reset_states()
        
        for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
            validation_step(model, token_ids, masks, labels)

        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):
            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
            validation_auc_metrics[i].reset_states()

        print('\n')

        
train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=NR_EPOCHS)

================================================== EPOCH 0 ==================================================



Train Step: 0, Loss: 0.9446811079978943
label0 roc_auc 0.1428571492433548
label1 roc_auc 0.5546875
label2 roc_auc 0.5333333015441895

Train Step: 1000, Loss: 0.5480075478553772
label0 roc_auc 0.7637864351272583
label1 roc_auc 0.6970726251602173
label2 roc_auc 0.5257648229598999

Train Step: 2000, Loss: 0.43559107184410095
label0 roc_auc 0.9141294956207275
label1 roc_auc 0.9106881618499756
label2 roc_auc 0.8555693030357361

Train Step: 3000, Loss: 0.3920212686061859
label0 roc_auc 0.9211728572845459
label1 roc_auc 0.9186181426048279
label2 roc_auc 0.8794183731079102

Train Step: 4000, Loss: 0.3683013916015625
label0 roc_auc 0.9281899929046631
label1 roc_auc 0.9241409301757812
label2 roc_auc 0.8818445801734924

Train Step: 5000, Loss: 0.3549581468105316
label0 roc_auc 0.9243957996368408
label1 roc_auc 0.9198724627494812
label2 roc_auc 0.8792656064033508

Train Step: 6000, Loss: 0.3451485335826874
label0 roc_auc 0.9269123673439026
label1 roc_auc 0.9245182871818542
label2 roc_auc 0.879790



Epoch 1, Validation Loss: 0.26953327655792236, Time: 20524.724361658096

label0 roc_auc 0.9403871893882751
label1 roc_auc 0.9374871850013733
label2 roc_auc 0.9072442054748535




In [ ]:
test_input_ids = tokenize_sentences(test['text'], tokenizer, MAX_LEN)
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_attention_masks = create_attention_masks(test_input_ids)

In [ ]:
test  = test.reset_index(drop=True)
test['ID'] = test.index

df_submission = pd.DataFrame({"ID":test['ID'],"label0":np.nan,"label1":np.nan,"label2":np.nan,})

In [ ]:
TEST_BATCH_SIZE = 16
test_steps = len(test) // TEST_BATCH_SIZE

test_dataset = create_dataset((test_input_ids, test_attention_masks), batch_size=TEST_BATCH_SIZE, train=False, epochs=1)

# df_submission = pd.read_csv("../input/moneykingdom/sample_submission_UVKGLZE.csv", index_col='ID')

for i, (token_ids, masks) in enumerate(tqdm(test_dataset, total=test_steps)):
    sample_ids = test.iloc[i*TEST_BATCH_SIZE:(i+1)*TEST_BATCH_SIZE]['ID']
    predictions = model(token_ids, attention_mask=masks).numpy()

    df_submission.loc[sample_ids, label_cols] = predictions

In [ ]:
df_submission.to_csv("bert_LARGE_result.csv")

In [ ]:
preds1 = np.array(df_submission[label_cols])

In [ ]:
preds1  = pd.DataFrame({'label':[i.argmax() for i in preds1]})

In [ ]:
preds1.value_counts()

label
1        14211
0        13919
2         1870
dtype: int64

In [ ]:
preds1.to_csv("Bert_Classification.csv")

In [ ]:
model.summary()

Model: "bert_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model (TFBertModel)  multiple                  335141888 
_________________________________________________________________
dense (Dense)                multiple                  3075      
Total params: 335,144,963
Trainable params: 335,144,963
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model.save_weights('/my_bert/bert')